# Regime II Hyperparameter Search

# Setup:

## Connect to Remote Compute Environment

First ensure we are connected to the correct VSCode Remote Kernel.

In [ ]:
!uname -nv && ls /

## Upgrade Python Modules

Install the latest version of Tensorflow, and install Tensorflow

In [ ]:
!pip3 install --quiet --upgrade tensorflow==2.11.0
!pip3 install --quiet tensorflow_addons

## Python Environment Checks

Instantiate Python Kernel and load Python modules.

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Attempt to dynamic GPU memory (vram) allocation
try:
    tf.config.experimental.set_memory_growth(
        tf.config.list_physical_devices('GPU')[0],
        enable=True
    )
except IndexError as e:
    print("No GPU detected. Dynamic GPU vRAM allocation failed.")
    
import tensorflow_addons as tfa
import keras
from keras import layers
from typing import Literal, Union, TypeVar

# Import utility functions defined in ../common/ package
import sys
sys.path.append('../')
from common import *

Double-check GPU is available.

In [ ]:
display(tf.__version__)
display(tf.config.list_physical_devices('GPU'))
display(tf.test.gpu_device_name())

try:
    tf.config.experimental.set_memory_growth(
        tf.config.list_physical_devices('GPU')[0],
        enable=True
    )
except IndexError as e:
    display("No GPU Found")

# Model Preparation

Begin preparing the model's execution environment. First, we start by defining some constants:

In [ ]:
IMG_SIZE  : tuple[int, int] = (299, 299)
AUTOTUNE  : Literal = tf.data.AUTOTUNE
RNG_SEED  : int = 1337

# For Remote
dataset_directory: str = "./"

# For Local
# dataset_directory: str = "../../dataset/"

## Prepare Datasets

We load the datasets which are by default made available as `tf.data.Dataset` objects using a train-test-validation split of 70%, 15%, 15%. Since we are using k-fold validation, we must first concatenate the training and validation set, which will be split later on using our k-fold validation routine.

In [ ]:
# InceptionV3 requires image tensors with a shape of (299, 299, 3) 
ds_train: tf.data.Dataset = tf.data.Dataset.load(dataset_directory + "ds_train")
ds_valid: tf.data.Dataset = tf.data.Dataset.load(dataset_directory + "ds_valid")
ds_test : tf.data.Dataset = tf.data.Dataset.load(dataset_directory + "ds_test")

# For K-Fold Cross Validation
ds_train_and_valid: tf.data.Dataset = ds_train.concatenate(ds_test)

# Batching, caching, and performance optimisations are *not* performed at this stage
# Since we are doing K-Fold validation

# configure_for_performance(ds_train)
# configure_for_performance(ds_valid)
# configure_for_performance(ds_test)

In [ ]:
preview_dataset(ds_train_and_valid)

# Regime II

## Hyperparameters Under Consideration
* Adam Optimizer:
    * Learning Rate: 0.01 to 0.001 (default) to 0.0001
        * 1.0e-1 to 1.0e-4
        * [1, 2, 3, 4]
    * epsilon: 0.00000001 to 0.1
        * exponential search values:
            * 1.0e-8 to 1.0e-1
            * [1, 2, 3, 4, 5, 6, 7, 8]


In [ ]:
BATCH_SIZE: int = 1600
DROPOUT_RATE: float = 0.2
EPOCHS    : int = 20
METRICS: list[any] = [
    tf.keras.metrics.AUC(multi_label=True, num_labels=18),
    tf.keras.metrics.Precision(thresholds=0.5),
    tf.keras.metrics.Recall(thresholds=0.5),
    tfa.metrics.F1Score(num_classes=18, average='macro', threshold=0.5),
]

In [ ]:
def learning_rate_gridsearch(
        kfolds: int = 6,
        filename: str = 'regime_II_search_results.pickle'
    ) -> list[dict[str, Union[int, float, list[tf.keras.callbacks.History]]]]:
    """
    Performs a grid search for hyperparameters 'learning_rate' and 'epsilon_rate' using K-Fold cross validation.

    Args:
        kfolds (int, optional): Number of folds for K-Fold cross validation. Defaults to 6.
        filename (str, optional): File name to save the search results. Defaults to 'regime_II_search_results.pickle'.

    Returns:
        list[dict[str, Union[int, float, list[tf.keras.callbacks.History]]]]: List of dictionaries containing the search results.
            Each dictionary contains the following keys:
                - 'learning_rate' (float): The learning rate hyperparameter used in the experiment.
                - 'epsilon_rate' (float): The epsilon rate hyperparameter used in the experiment.
                - 'history_list' (list[tf.keras.callbacks.History]): List of Keras History objects containing training history for each fold in K-Fold cross validation.
    """
    
    # Define the hyperparameter search grid
    learning_rates: list = [1.0 * np.float_power(10, -rate) for rate in range(1, 5)]
    epsilon_rates : list = [1.0 * np.float_power(10, -rate) for rate in range(1, 9)]

    search_results: list[dict[str, Union[int, float, list[tf.keras.callbacks.History]]]] = []
    for i, learning_rate in enumerate(learning_rates):
        for j, epsilon_rate in enumerate(epsilon_rates):
            index: int = (i * len(epsilon_rates)) + j
            print(f"\n### Grid Search {index + 1}/{len(epsilon_rates) * len(learning_rates)}: learning_rate: {np.format_float_scientific(learning_rate)}, epsilon_rate: {np.format_float_scientific(epsilon_rate)} ###")

            # Conduct K-Fold Experiment
            k_fold_results: list[tf.keras.callbacks.History] = cross_validate(
                TransferLearningModel,
                ds_train_and_valid,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                k=kfolds,
                optimizer_kwargs={"learning_rate": learning_rate, "epsilon": epsilon_rate},
                model_kwargs={"dropout_rate": DROPOUT_RATE}
            )

            search_results.append({
                "learning_rate": learning_rate,
                "epsilon_rate" : epsilon_rate,
                "history_list" : k_fold_results
            })

            # Save results in case hyperparameter search gets interrupted
            with open(filename, 'wb') as file:
                pickle.dump(search_results, file, protocol=pickle.HIGHEST_PROTOCOL)

    print("ALL DONE")
    return search_results

## Begin Search

This will take a long time.

In [ ]:
learning_rate_gridsearch()